In [ ]:
#Sin esa condición del átomo que sale de la trampa
#Borrador de mi codigo con correciones
#Le alta estructura a la trampa para el ion en relación al
import numpy as np
import math
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import random
from mpl_toolkits.mplot3d import Axes3D
from multiprocessing import Pool, cpu_count

#Ion: Yb+ , atom: Li

#constants
m_ion = 2.8733965E-25  #(YB+) 2,8733965 × 10^-25 kg (SR+ 2.83E-25)
m_aton = 1.1525801E-25 # (LITIO) 1,1525801 × 10^-26 kg (RB 1.44E-25 )
m_atom = 1.1525801E-25 # (LITIO) 1,1525801 × 10^-26 kg (RB 1.44E-25 )
C4=5.607E-57   #C4=5.607E-57   Paper Prospects #C4=1.1E-56 RbSr+
C6=5E-19 * C4   #C6=5.0E-19     Paper Prospects (LIYB+) C6 = 2.803 × 10−75 Jm6 C4 = 2.803 × 10−57
C8= 4.49088772E-76  #Coeffiente Vander Walls Rb_Rb originally 4691 ao
#frf = 2.5 * 10**6
Omega= 2.0 * np.pi * 10**6 #MHz #Depende tambien mucho de esta fase
a = -2.982E-4  #-3E-7, da un buen comportamiento
q = 0.2      #Entre 0.1 y otro
T_ion =  0 # da un buen comportamiento  0.5 milikelvin   Paper Ziv Meir (de Jesus)
T_atom = 0.1E-6  #0.1E-6, da un buen comportamiento   #6 microkelvin (mucho más frio átomo que ion) Paper Ziv Meir
r0 = 0.1E-6
ti= 0.0
tf= 200E-6          #0.001, da un buen comportamiento pal ion #de matlab 200e-6;
dt= 10000
num_solution = 1
num_collisions = 10
phi = 2 * np.pi * np.random.rand()
theta = np.arccos(2 * np.random.rand() - 1)
ptol = 1e-10

def system(t, y, a, q, Omega, C4, C6, m_ion, m_a):

    ionx, iony, ionz, ionvx, ionvy, ionvz, atomx, atomy, atomz, atomvx, atomvy, atomvz = y

    # Definición de las ecuaciones diferenciales
    dydt = np.zeros_like(y)

    #Velocidades del ion
    dydt[0] = ionvx
    dydt[1] = ionvy
    dydt[2] = ionvz

    #velocidades del átomo
    dydt[6] = atomvx
    dydt[7] = atomvy
    dydt[8] = atomvz

    #Potencial para el ion
    dydt[3] = -(a+2*q*np.cos(Omega*t))*ionx*(Omega**2 / 4)
    dydt[4] = -(a+2*q*np.cos(Omega*t))*iony*(Omega**2 / 4)
    dydt[5] = -(2*a+2*-q*np.cos(Omega*t))*ionz*(Omega**2 / 4)

    #Interacción con el potencial atom
    r = np.sqrt((atomx - ionx)**2 + (atomy - iony)**2 + (atomz - ionz)**2)

    dydt[3] += -(2*C4/r**5 -6 *C6 / r**7) * (ionx - atomx) / (m_ion * r)
    dydt[4] += -(2*C4/r**5 -6 *C6 / r**7) * (iony - atomy) / (m_ion * r)
    dydt[5] += -(2*C4/r**5 -6 *C6 / r**7) * (ionz - atomz) / (m_ion * r)

    dydt[9] += -(2*C4/r**5 -6 *C6 / r**7) * (atomx - ionx) / (m_atom * r)
    dydt[10] += -(2*C4/r**5 -6 *C6 / r**7) * (atomy - iony) / (m_atom * r)
    dydt[11] += -(2*C4/r**5 -6 *C6 / r**7) * (atomz - ionz) / (m_atom * r)

    return dydt

def system2(t, y, a, q, Omega, C4, C6, C8, m_ion, m_a):

    ionx, iony, ionz, ionvx, ionvy, ionvz, atomx, atomy, atomz, atomvx, atomvy, atomvz, atonx, atony, atonz, atonvx, atonvy, atonvz = y

    # Definición de las ecuaciones diferenciales
    dydt = np.zeros_like(y)

    #Velocidades del ion
    dydt[0] = ionvx
    dydt[1] = ionvy
    dydt[2] = ionvz

    #velocidades del átomo
    dydt[6] = atomvx
    dydt[7] = atomvy
    dydt[8] = atomvz

    #Potencial para el ion
    dydt[3] = -(a+2*q*np.cos(Omega*t))*ionx*(Omega**2 / 4)
    dydt[4] = -(a+2*q*np.cos(Omega*t))*iony*(Omega**2 / 4)
    dydt[5] = -(2*a+2*-q*np.cos(Omega*t))*ionz*(Omega**2 / 4)

    #Interacción con el potencial atom
    r = np.sqrt((atomx - ionx)**2 + (atomy - iony)**2 + (atomz - ionz)**2)

    dydt[3] += -(2*C4/r**5 -6 *C6 / r**7) * (ionx - atomx) / (m_ion * r)
    dydt[4] += -(2*C4/r**5 -6 *C6 / r**7) * (iony - atomy) / (m_ion * r)
    dydt[5] += -(2*C4/r**5 -6 *C6 / r**7) * (ionz - atomz) / (m_ion * r)

    dydt[9] += -(2*C4/r**5 -6 *C6 / r**7) * (atomx - ionx) / (m_atom * r)
    dydt[10] += -(2*C4/r**5 -6 *C6 / r**7) * (atomy - iony) / (m_atom * r)
    dydt[11] += -(2*C4/r**5 -6 *C6 / r**7) * (atomz - ionz) / (m_atom * r)

    dydt[12] = atonvx
    dydt[13] = atonvy
    dydt[14] = atonvz

    #Interacción con el potencial atom-atom
    r2 = np.sqrt((atonx - atomx)**2 + (atony - atomy)**2 + (atonz - atomz)**2)

    dydt[9] += -(6*C8/r2**7) * (atomx - atonx) / (m_a * r2)
    dydt[10] += -(6*C8/r2**7) * (atomy - atony) / (m_a * r2)
    dydt[11] += -(6*C8/r2**7) * (atomz - atonz) / (m_a * r2)

    dydt[15] += -(6 *C8/r2**7) * (atonx - atomx) / (m_a * r2)
    dydt[16] += -(6 *C8/r2**7) * (atony - atomy) / (m_a * r2)
    dydt[17] += -(6 *C8/r2**7) * (atonz - atomz) / (m_a * r2)

    return dydt

#Funciones
def position_on_sphere(radius):
    # Convertir a coordenadas cartesianas
    x = radius * np.sin(theta) * np.cos(phi)
    y = radius * np.sin(theta) * np.sin(phi)
    z = radius * np.cos(theta)
    return x, y, z

def velocidad_atom(temperature, mass, x, y, z):

    kb = 1.380649e-23  # Constante de Boltzmann en m^2 kg s^-2 K^-1

    # Spherical velocities
    v_theta = np.sqrt((2 * kb * temperature) / mass) * random.gauss(0, 1)
    v_phi = np.sqrt((2 * kb * temperature) / mass) * random.gauss(0, 1)
    v_r = -abs(np.random.weibull(2) * np.sqrt(2 * kb * temperature / mass))

    # Componentes de velocidad en coordenadas cartesianas
    v_x = v_r * np.sin(theta) * np.cos(phi) + v_theta * np.cos(theta) * np.cos(phi) - v_phi * np.sin(phi)
    v_y = v_r * np.sin(theta) * np.sin(phi) + v_theta * np.cos(theta) * np.sin(phi) + v_phi * np.cos(phi)
    v_z = v_r * np.cos(theta) - v_theta * np.sin(theta)

    return v_x, v_y, v_z

def velocidad_ion(temperature, mass):
    kb = 1.38064E-23  # Constante de Boltzmann en m^2 g s^-2 K^-1, kelvin, metros, segundos, kilogramos
    v = np.sqrt((2 * kb * temperature) / mass) * random.gauss(0, 1)
    return v

#Colision se detiene si átomo sale de segunda esfera de radio 2E-7
def event_condition(t, y, a, q, Omega, C4, C6, m_ion, m_a):
    # Definir condición de evento
    ionx, iony, ionz, ionvx, ionvy, ionvz, atomx, atomy, atomz, atomvx, atomvy, atomvz = y
    s = np.sqrt((atomx)**2 + (atomy)**2 + (atomz)**2)
    return s - 2E-7

# Indicar que el evento se debe buscar a través del cruce con cero
event_condition.terminal = True
event_condition.direction = 1

# Definición del vector de tiempo
t_span = (ti, tf)

# Resolución del sistema de ecuaciones diferenciales
def run_simulation(i):
    results = []

    for collision in range (num_collisions):
        #inicialization variables
        ionx_0=0.0
        iony_0=0.0
        ionz_0=0.0
        ionvx_0=velocidad_ion(T_ion, m_ion)
        ionvy_0=velocidad_ion(T_ion, m_ion)
        ionvz_0=velocidad_ion(T_ion, m_ion)
        atomx_0, atomy_0, atomz_0 = position_on_sphere(r0)
        atomvx_0, atomvy_0, atomvz_0 = velocidad_atom(T_atom, m_atom, atomx_0, atomy_0, atomz_0)
        #Creo que debo ponerlo aqui xdd
        atonx_0, atony_0, atonz_0 = position_on_sphere(r0)
        atonvx_0, atonvy_0, atonvz_0 = velocidad_atom(T_atom, m_atom, atonx_0, atony_0, atonz_0)

        y0 = [ionx_0, iony_0, ionz_0, ionvx_0, ionvy_0, ionvz_0, atomx_0, atomy_0, atomz_0, atomvx_0, atomvy_0, atomvz_0]
        solution = solve_ivp(system, t_span, y0, t_eval=np.linspace(t_span[0], t_span[1], dt), method='RK45', args=(a, q, Omega, C4, C6, m_ion, m_atom), rtol=ptol, events= event_condition)

        # Extracción de las soluciones temporales
        t_sol = solution.t
        ionx_sol = solution.y[0]
        iony_sol = solution.y[1]
        ionz_sol = solution.y[2]
        ionvx_sol = solution.y[3]
        ionvy_sol = solution.y[4]
        ionvz_sol = solution.y[5]
        atomx_sol = solution.y[6]
        atomy_sol = solution.y[7]
        atomz_sol = solution.y[8]
        atomvx_sol = solution.y[9]
        atomvy_sol = solution.y[10]
        atomvz_sol = solution.y[11]

        results.append([t_sol, ionx_sol, iony_sol, ionz_sol, ionvx_sol, ionvy_sol, ionvz_sol, atomx_sol, atomy_sol, atomz_sol, atomvx_sol, atomvy_sol, atomvz_sol])

    new_results = []
    #Initial positions and velocities for other atom
    atonx_0, atony_0, atonz_0 = position_on_sphere(r0)
    atonvx_0, atony_0, atonz_0 = velocidad_atom(T_atom, m_atom, atonx_0, atony_0, atonz_0)
    t_span2= (0.0, 200E-6)
    y02 = [[ionx_sol[-1]], [iony_sol[-1]], [ionz_sol[-1], [ionvx_sol[-1]], [ionvy_sol[-1]], [ionvz_sol[-1]] ], atomx_sol[-1], atomy_sol[-1], atomz_sol[-1], atomvx_sol[-1], atomvy_sol[-1], atomvz_sol[-1], atonx_0, atony_0, atonz_0, atonvx_0, atonvy_0, atonvz_0]
    solution2 = solve_ivp(system2, t_span2, y02, t_eval2=np.linspace(t_span2[0], t_span2[1], dt), method='RK45', args=(a, q, Omega, C4, C6, C8, m_ion, m_atom), rtol=ptol)

    t_sol2 = solution2.t
    ionx_sol2 = solution2.y[0]
    iony_sol2 = solution2.y[1]
    ionz_sol2 = solution2.y[2]
    ionvx_sol2 = solution2.y[3]
    ionvy_sol2 = solution2.y[4]
    ionvz_sol2 = solution2.y[5]
    atomx_sol2 = solution2.y[6]
    atomy_sol2 = solution2.y[7]
    atomz_sol2 = solution2.y[8]
    atomvx_sol2 = solution2.y[9]
    atomvy_sol2 = solution2.y[10]
    atomvz_sol2 = solution2.y[11]
    atonx_sol2 = solution2.y[12]
    atony_sol2 = solution2.y[13]
    atonz_sol2 = solution2.y[14]
    atonvx_sol2 = solution2.y[15]
    atonvy_sol2 = solution2.y[16]
    atonvz_sol2 = solution2.y[17]


    #Se actulizan las condiciones para el ion y el atomo
    #y0 = [ionx_0, iony_0, ionz_0, ionvx_0, ionvy_0, ionvz_0, atomx_0, atomy_0, atomz_0, atomvx_0, atomvy_0, atomvz_0]

    return results


if __name__ == '__main__':
    num_solution = 1
    #num_cpus = cpu_count()
    num_cpus = 1
    with Pool(processes=num_cpus) as pool:
        all_results = pool.map(run_simulation, range(num_solution))
    #print(all_results)

    for results in all_results:
        Ekin_total = []
        for result in results:
            t_sol=result[0]
            ionx_sol=result[1]
            iony_sol=result[2]
            ionz_sol=result[3]
            ionvx_sol=result[4]
            ionvy_sol=result[5]
            ionvz_sol=result[6]
            atomx_sol=result[7]
            atomy_sol=result[8]
            atomz_sol=result[9]
            atomvx_sol=result[10]
            atomvy_sol=result[11]
            atomvz_sol=result[12]


            # Crear una nueva figura con subplots para ion y átomo
            plt.plot(ionx_sol, iony_sol, label='x vs y - ion')
            plt.plot(atomx_sol, atomy_sol, label='x vs y - aton')
            plt.xlabel('X')
            plt.ylabel('Y')
            plt.legend()
            plt.grid()
            plt.show()


            #pruebas
            #print(ionx_sol[0], iony_sol[0], ionz_sol[0])
            #print(ionx_sol[-1], iony_sol[-1], ionz_sol[-1])
            #print(ionvx_sol[0], ionvy_sol[0], ionvz_sol[0])
            #print(ionvx_sol[-1], ionvy_sol[-1], ionvz_sol[-1])
            #print(Ekin_total)
            #print(np.sqrt((atomx_sol)**2 + (atomy_sol)**2 + (atomz_sol)**2))
            #print (atomx_sol[0], atomy_sol[0], atomx_sol[-1], atomy_sol[-1])
            #print(t_sol, E_kiin, E_kiin_mean)


    #plt.plot(Ekin_total)
    #print(Ekin_total)
    #plt.xlabel('Number of collisions')
    #plt.ylabel('Average kinetic energy')
    #plt.grid()
    #plt.show()

/usr/local/lib/python3.10/dist-packages/scipy/integrate/_ivp/common.py:39: UserWarning: The following arguments have no effect for a chosen solver: `t_eval2`.
  warn("The following arguments have no effect for a chosen solver: {}."


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (15,) + inhomogeneous part.